In [3]:
# Install system dependencies
!apt-get update
!apt-get install -y libcairo2-dev pkg-config python3-dev
!pip install flask flask-cors pyngrok pandas diffusers torch transformers pillow reportlab cairosvg psutil googletrans==3.1.0a0
print("Done")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [4]:
import os
import pandas as pd
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image, ImageDraw, ImageFont
from googletrans import Translator
import time
import psutil
from transformers import CLIPProcessor, CLIPModel
import uuid
from werkzeug.utils import secure_filename
import threading
import atexit
import base64
import io

# --- Setup ---
os.makedirs('uploads', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

app = Flask(__name__)
CORS(app)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['OUTPUT_FOLDER'] = 'outputs'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# --- Initialize Models ---
try:
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    models_loaded = True
except Exception as e:
    print(f"Model loading error: {e}")
    pipe = None
    clip_model = None
    clip_processor = None
    models_loaded = False

translator = Translator()

# --- Helper Functions ---
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() == 'csv'

def pil_image_to_base64(image):
    """Convert PIL Image to base64 string"""
    try:
        buffer = io.BytesIO()
        # Ensure image is in RGB mode for better compatibility
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image.save(buffer, format='PNG')
        buffer.seek(0)
        img_bytes = buffer.getvalue()
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        print(f"PIL to Base64 conversion error: {e}")
        return None

def image_file_to_base64(image_path):
    """Convert image file to base64 string"""
    try:
        with open(image_path, "rb") as image_file:
            img_bytes = image_file.read()
            img_base64 = base64.b64encode(img_bytes).decode('utf-8')
            return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        print(f"File to Base64 conversion error: {e}")
        return None

def detect_column_types(df):
    """Detect numeric and categorical columns"""
    numeric_cols = []
    categorical_cols = []

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            numeric_cols.append(col)
        else:
            categorical_cols.append(col)

    return numeric_cols, categorical_cols

def generate_chart_config(data, chart_type, x_col, y_col, color_scheme, limit=10):
    """Generate chart configuration for frontend"""
    limited_data = data[:limit]

    labels = [str(row[x_col]) for row in limited_data]
    values = []

    for row in limited_data:
        try:
            values.append(float(row[y_col]))
        except:
            values.append(0)

    color_map = {
        "blue-green": {"bg": "rgba(0, 128, 128, 0.7)", "border": "rgba(0, 128, 128, 1)"},
        "blue": {"bg": "rgba(0, 100, 255, 0.7)", "border": "rgba(0, 100, 255, 1)"},
        "neutral": {"bg": "rgba(100, 100, 100, 0.7)", "border": "rgba(100, 100, 100, 1)"}
    }

    colors = color_map.get(color_scheme, color_map["blue"])

    return {
        'type': chart_type,
        'data': {
            'labels': labels,
            'datasets': [{
                'label': f'{y_col} by {x_col}',
                'data': values,
                'backgroundColor': colors["bg"],
                'borderColor': colors["border"],
                'borderWidth': 2
            }]
        },
        'options': {
            'responsive': True,
            'scales': {
                'y': {'beginAtZero': True}
            } if chart_type in ['bar', 'line'] else {}
        }
    }

def generate_infographic_prompt(data, x_col, y_col, title, tone, color_scheme, custom_prompt=None):
    """Generate AI prompt for infographic creation"""
    if custom_prompt:
        return custom_prompt

    # Analyze data for better prompts
    data_summary = f"data about {x_col} and {y_col}"
    if len(data) > 0:
        sample_values = [str(row[x_col]) for row in data[:3]]
        data_summary = f"data showing {x_col} categories: {', '.join(sample_values)}"

    tone_styles = {
        "formal": "professional, clean, corporate style",
        "casual": "friendly, approachable, modern style",
        "promotional": "eye-catching, vibrant, marketing style"
    }

    color_prompts = {
        "blue-green": "blue and teal color scheme, ocean-inspired colors",
        "blue": "blue color palette, professional blue tones",
        "neutral": "neutral colors, grayscale with subtle accents"
    }

    style = tone_styles.get(tone, "clean, professional style")
    colors = color_prompts.get(color_scheme, "blue color scheme")

    prompt = f"Modern infographic design, {style}, {colors}, data visualization, charts and graphs, {data_summary}, clean layout, white background, professional typography, high quality, detailed"

    return prompt

def create_infographic_overlay(base_image, data, x_col, y_col, title, tone, color_scheme, language):
    """Add data overlay to generated infographic and return PIL Image"""
    try:
        # Work with the PIL image directly
        image = base_image.convert("RGBA")
        draw = ImageDraw.Draw(image)

        try:
            font = ImageFont.truetype("arial.ttf", 20)
            title_font = ImageFont.truetype("arial.ttf", 24)
        except:
            font = ImageFont.load_default()
            title_font = ImageFont.load_default()

        color_map = {
            "blue-green": (0, 128, 128),
            "blue": (0, 0, 255),
            "neutral": (100, 100, 100)
        }
        text_color = color_map.get(color_scheme, (0, 0, 255))

        # Add title
        draw.text((50, 30), title, font=title_font, fill=text_color)

        # Add data points
        y_offset = 80
        for i, row in enumerate(data[:5]):  # Show top 5 data points
            text = f"{row[x_col]}: {row[y_col]}"
            draw.text((50, y_offset), text, font=font, fill=text_color)
            y_offset += 35

        return image
    except Exception as e:
        print(f"Overlay error: {e}")
        return None

# --- API Routes ---

@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'models_loaded': models_loaded,
        'timestamp': time.time()
    })

@app.route('/api/upload', methods=['POST'])
def upload_csv():
    """Upload and analyze CSV file"""
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file provided'}), 400

        file = request.files['file']
        if file.filename == '' or not allowed_file(file.filename):
            return jsonify({'error': 'Invalid file type. Only CSV files allowed'}), 400

        # Save file
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Analyze CSV
        df = pd.read_csv(filepath)
        numeric_cols, categorical_cols = detect_column_types(df)

        # Get sample data
        sample_data = df.head(5).to_dict(orient='records')

        return jsonify({
            'success': True,
            'filename': filename,
            'columns': df.columns.tolist(),
            'numeric_columns': numeric_cols,
            'categorical_columns': categorical_cols,
            'row_count': len(df),
            'sample_data': sample_data
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/chart-options', methods=['POST'])
def get_chart_options():
    """Get available chart options based on uploaded data"""
    try:
        data = request.get_json()
        filename = data.get('filename')

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        numeric_cols, categorical_cols = detect_column_types(df)

        # Suggest defaults
        x_default = categorical_cols[0] if categorical_cols else df.columns[0]
        y_default = numeric_cols[0] if numeric_cols else (df.columns[1] if len(df.columns) > 1 else df.columns[0])

        options = {
            'chart_types': [
                {'value': 'bar', 'label': 'Bar Chart'},
                {'value': 'line', 'label': 'Line Chart'},
                {'value': 'pie', 'label': 'Pie Chart'},
                {'value': 'infographic', 'label': 'AI Infographic'}
            ],
            'columns': df.columns.tolist(),
            'tones': [
                {'value': 'formal', 'label': 'Formal'},
                {'value': 'casual', 'label': 'Casual'},
                {'value': 'promotional', 'label': 'Promotional'}
            ],
            'color_schemes': [
                {'value': 'blue-green', 'label': 'Blue Green'},
                {'value': 'blue', 'label': 'Blue'},
                {'value': 'neutral', 'label': 'Neutral'}
            ],
            'languages': [
                {'value': 'en', 'label': 'English'},
                {'value': 'hi', 'label': 'Hindi'}
            ],
            'defaults': {
                'chart_type': 'bar',
                'x_axis': x_default,
                'y_axis': y_default,
                'tone': 'formal',
                'color_scheme': 'blue-green',
                'language': 'en'
            }
        }

        return jsonify({'success': True, 'options': options})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/generate', methods=['POST'])
def generate_visualization():
    """Generate chart or infographic based on parameters"""
    try:
        data = request.get_json()
        filename = data.get('filename')
        params = data.get('parameters', {})

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        # Load data
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        data_records = df.to_dict(orient='records')

        # Extract parameters
        chart_type = params.get('chart_type', 'bar')
        x_col = params.get('x_axis')
        y_col = params.get('y_axis')
        tone = params.get('tone', 'formal')
        color_scheme = params.get('color_scheme', 'blue-green')
        language = params.get('language', 'en')
        title = params.get('title', f'{y_col} by {x_col}')

        start_time = time.time()
        result = {'success': True}

        if chart_type == 'infographic' and models_loaded:
            # Generate AI infographic
            custom_prompt = params.get('custom_prompt')

            prompt = generate_infographic_prompt(
                data_records, x_col, y_col, title, tone, color_scheme, custom_prompt
            )

            print(f"🎨 Generated prompt: {prompt}")

            # Generate base image
            base_image = pipe(
                prompt,
                num_inference_steps=params.get('inference_steps', 25),
                guidance_scale=params.get('guidance_scale', 7.5),
                width=params.get('width', 512),
                height=params.get('height', 512)
            ).images[0]

            # Add data overlay
            final_image = create_infographic_overlay(
                base_image, data_records, x_col, y_col, title, tone, color_scheme, language
            )

            if final_image:
                # Convert to base64
                base64_image = pil_image_to_base64(final_image)

                if base64_image:
                    result['infographic'] = {
                        'base64': base64_image,
                        'prompt_used': prompt,
                        'format': 'PNG',
                        'dimensions': f"{final_image.width}x{final_image.height}"
                    }
                else:
                    return jsonify({'error': 'Failed to convert infographic to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate infographic'}), 500

        else:
            # Generate chart configuration
            chart_config = generate_chart_config(
                data_records, chart_type, x_col, y_col, color_scheme
            )
            result['chart_config'] = chart_config


        # Add metrics
        result['metrics'] = {
            'generation_time': round(time.time() - start_time, 2),
            'cpu_usage': psutil.cpu_percent(),
            'memory_usage': psutil.virtual_memory().percent
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/generate-base64-only', methods=['POST'])
def generate_base64_only():
    """Generate visualization and return only base64 (no file saving)"""
    try:
        data = request.get_json()
        filename = data.get('filename')
        params = data.get('parameters', {})

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        # Load data
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        data_records = df.to_dict(orient='records')

        # Extract parameters
        chart_type = params.get('chart_type', 'infographic')
        x_col = params.get('x_axis')
        y_col = params.get('y_axis')
        tone = params.get('tone', 'formal')
        color_scheme = params.get('color_scheme', 'blue-green')
        title = params.get('title', f'{y_col} by {x_col}')

        start_time = time.time()

        if chart_type == 'infographic' and models_loaded:
            # Generate AI infographic
            custom_prompt = params.get('custom_prompt')

            prompt = generate_infographic_prompt(
                data_records, x_col, y_col, title, tone, color_scheme, custom_prompt
            )

            print(f"🎨 Generated prompt: {prompt}")

            # Generate base image
            base_image = pipe(
                prompt,
                num_inference_steps=params.get('inference_steps', 25),
                guidance_scale=params.get('guidance_scale', 7.5),
                width=params.get('width', 512),
                height=params.get('height', 512)
            ).images[0]

            # Add data overlay
            final_image = create_infographic_overlay(
                base_image, data_records, x_col, y_col, title, tone, color_scheme, 'en'
            )

            if final_image:
                # Convert to base64
                base64_image = pil_image_to_base64(final_image)

                if base64_image:
                    return jsonify({
                        'success': True,
                        'image_base64': base64_image,
                        'prompt_used': prompt,
                        'format': 'PNG',
                        'dimensions': f"{final_image.width}x{final_image.height}",
                        'metrics': {
                            'generation_time': round(time.time() - start_time, 2),
                            'cpu_usage': psutil.cpu_percent(),
                            'memory_usage': psutil.virtual_memory().percent
                        }
                    })
                else:
                    return jsonify({'error': 'Failed to convert image to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate infographic'}), 500
        else:
            return jsonify({'error': 'Infographic generation not available'}), 400

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/convert-to-base64', methods=['POST'])
def convert_existing_to_base64():
    """Convert existing saved image to base64"""
    try:
        data = request.get_json()
        filename = data.get('filename')

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        filepath = os.path.join(app.config['OUTPUT_FOLDER'], filename)

        if not os.path.exists(filepath):
            return jsonify({'error': 'File not found'}), 404

        base64_image = image_file_to_base64(filepath)

        if base64_image:
            return jsonify({
                'success': True,
                'image_base64': base64_image,
                'format': 'PNG'
            })
        else:
            return jsonify({'error': 'Failed to convert image to base64'}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/outputs/<filename>')
def serve_output(filename):
    """Serve generated output files (fallback for direct file access)"""
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename)

@app.route('/api/data/<filename>')
def get_data_preview(filename):
    """Get data preview for a specific file"""
    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)

        # Get basic stats
        stats = {
            'shape': df.shape,
            'columns': df.columns.tolist(),
            'dtypes': df.dtypes.to_dict(),
            'null_counts': df.isnull().sum().to_dict(),
            'sample_data': df.head(10).to_dict(orient='records')
        }

        # Add numeric column statistics
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            stats['numeric_stats'] = df[numeric_cols].describe().to_dict()

        return jsonify({'success': True, 'stats': stats})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/prompt-suggestions', methods=['POST'])
def get_prompt_suggestions():
    """Get AI prompt suggestions based on data"""
    try:
        data = request.get_json()
        x_col = data.get('x_axis')
        y_col = data.get('y_axis')
        tone = data.get('tone', 'formal')
        color_scheme = data.get('color_scheme', 'blue')
        data_type = data.get('data_type', 'general')

        suggestions = []

        # Base prompts by data type
        base_prompts = {
            'sales': f"Professional sales infographic showing {x_col} vs {y_col}, corporate style, charts and graphs",
            'marketing': f"Eye-catching marketing infographic, {x_col} and {y_col} data, vibrant design, call-to-action elements",
            'analytics': f"Clean analytics dashboard style, {x_col} by {y_col}, data visualization, KPI layout",
            'financial': f"Financial report infographic, {x_col} and {y_col} metrics, professional charts, business style",
            'general': f"Modern data infographic showing {x_col} vs {y_col}, clean layout, professional design"
        }

        # Add tone and color variations
        for data_cat, base in base_prompts.items():
            prompt = generate_infographic_prompt([], x_col, y_col, f"{x_col} Analysis", tone, color_scheme)
            suggestions.append({
                'category': data_cat,
                'prompt': prompt,
                'description': f"{data_cat.title()} style infographic"
            })

        return jsonify({
            'success': True,
            'suggestions': suggestions,
            'custom_parameters': {
                'inference_steps': {'min': 10, 'max': 50, 'default': 25},
                'guidance_scale': {'min': 1, 'max': 20, 'default': 7.5},
                'width': {'options': [512, 768, 1024], 'default': 512},
                'height': {'options': [512, 768, 1024], 'default': 512}
            }
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# --- Ngrok Configuration ---
NGROK_AUTH_TOKEN = "30JBYOiedMdItKFKaBrSQcQ82yo_4EtXeU17QiqfTpDAYSieA"
ngrok_tunnel = None

def setup_ngrok():
    """Setup ngrok tunnel"""
    global ngrok_tunnel
    try:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        ngrok_tunnel = ngrok.connect(5000)
        public_url = ngrok_tunnel.public_url
        print(f"🌐 Public URL: {public_url}")
        print(f"🔗 API Base URL: {public_url}/api")
        return public_url
    except Exception as e:
        print(f"❌ Ngrok setup failed: {e}")
        return None

def cleanup_ngrok():
    """Cleanup ngrok tunnel on shutdown"""
    global ngrok_tunnel
    try:
        if ngrok_tunnel:
            ngrok.disconnect(ngrok_tunnel.public_url)
        ngrok.kill()
        print("🔌 Ngrok tunnel closed")
    except:
        pass

# Register cleanup function
atexit.register(cleanup_ngrok)

@app.route('/api/tunnel-info', methods=['GET'])
def get_tunnel_info():
    """Get ngrok tunnel information"""
    global ngrok_tunnel
    if ngrok_tunnel:
        return jsonify({
            'success': True,
            'public_url': ngrok_tunnel.public_url,
            'api_base_url': f"{ngrok_tunnel.public_url}/api",
            'status': 'active'
        })
    else:
        return jsonify({
            'success': False,
            'status': 'inactive',
            'message': 'Ngrok tunnel not active'
        })

# --- Error Handlers ---
@app.errorhandler(413)
def too_large(e):
    return jsonify({'error': 'File too large. Maximum size is 16MB'}), 413

@app.errorhandler(404)
def not_found(e):
    return jsonify({'error': 'Endpoint not found'}), 404

@app.errorhandler(500)
def internal_error(e):
    return jsonify({'error': 'Internal server error'}), 500

# --- Run App ---
if __name__ == '__main__':
    print("🚀 Starting AI Infographic API Server...")
    print(f"🤖 Models loaded: {models_loaded}")

    # Setup ngrok tunnel
    public_url = setup_ngrok()

    print("\n📋 Available endpoints:")
    print("  POST /api/upload - Upload CSV file")
    print("  POST /api/chart-options - Get chart options")
    print("  POST /api/generate - Generate visualization (includes base64)")
    print("  POST /api/generate-base64-only - Generate only base64 image")
    print("  POST /api/convert-to-base64 - Convert existing file to base64")
    print("  POST /api/prompt-suggestions - Get AI prompt suggestions")
    print("  GET /api/health - Health check")
    print("  GET /api/tunnel-info - Ngrok tunnel info")
    print("  GET /api/data/<filename> - Data preview")
    print("  GET /api/outputs/<filename> - Serve generated files (fallback)")

    if public_url:
        print(f"\n🌍 Your API is accessible at: {public_url}/api")
        print(f"📖 Example: {public_url}/api/health")

    print("\n⚡ Server starting on http://localhost:5000")
    print("🔄 Use Ctrl+C to stop the server")

    try:
        app.run(host='0.0.0.0', port=5000, debug=False)
    except KeyboardInterrupt:
        print("\n🛑 Server stopped by user")
    except Exception as e:
        print(f"❌ Server error: {e}")
    finally:
        cleanup_ngrok()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
import os
import pandas as pd
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image, ImageDraw, ImageFont
from googletrans import Translator
import time
import psutil
from transformers import CLIPProcessor, CLIPModel
import uuid
from werkzeug.utils import secure_filename
import threading
import atexit
import base64
import io
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import numpy as np

# --- Setup ---
os.makedirs('uploads', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

app = Flask(__name__)
CORS(app)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['OUTPUT_FOLDER'] = 'outputs'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# --- Initialize Models ---
try:
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    models_loaded = True
except Exception as e:
    print(f"Model loading error: {e}")
    pipe = None
    clip_model = None
    clip_processor = None
    models_loaded = False

translator = Translator()

# --- Helper Functions ---
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() == 'csv'

def pil_image_to_base64(image):
    """Convert PIL Image to base64 string"""
    try:
        buffer = io.BytesIO()
        # Ensure image is in RGB mode for better compatibility
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image.save(buffer, format='PNG')
        buffer.seek(0)
        img_bytes = buffer.getvalue()
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        print(f"PIL to Base64 conversion error: {e}")
        return None

def matplotlib_to_base64(fig):
    """Convert matplotlib figure to base64 string"""
    try:
        buffer = io.BytesIO()
        fig.savefig(buffer, format='PNG', dpi=300, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        buffer.seek(0)
        img_bytes = buffer.getvalue()
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        plt.close(fig)  # Close figure to free memory
        return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        print(f"Matplotlib to Base64 conversion error: {e}")
        return None

def image_file_to_base64(image_path):
    """Convert image file to base64 string"""
    try:
        with open(image_path, "rb") as image_file:
            img_bytes = image_file.read()
            img_base64 = base64.b64encode(img_bytes).decode('utf-8')
            return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        print(f"File to Base64 conversion error: {e}")
        return None

def detect_column_types(df):
    """Detect numeric and categorical columns"""
    numeric_cols = []
    categorical_cols = []

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            numeric_cols.append(col)
        else:
            categorical_cols.append(col)

    return numeric_cols, categorical_cols

def get_color_palette(color_scheme):
    """Get color palette based on scheme"""
    palettes = {
        "blue-green": ['#008080', '#20B2AA', '#40E0D0', '#00CED1', '#48D1CC'],
        "blue": ['#0066FF', '#3366FF', '#6699FF', '#0080FF', '#4DA6FF'],
        "neutral": ['#666666', '#808080', '#999999', '#B3B3B3', '#CCCCCC']
    }
    return palettes.get(color_scheme, palettes["blue"])

def create_chart_image(data, chart_type, x_col, y_col, title, color_scheme, limit=10):
    """Create chart image using matplotlib and return as PIL Image"""
    try:
        # Limit data
        limited_data = data[:limit]

        # Prepare data
        labels = [str(row[x_col]) for row in limited_data]
        values = []

        for row in limited_data:
            try:
                values.append(float(row[y_col]))
            except:
                values.append(0)

        # Get colors
        colors = get_color_palette(color_scheme)

        # Create figure
        fig, ax = plt.subplots(figsize=(12, 8))
        fig.patch.set_facecolor('white')

        if chart_type == 'bar':
            bars = ax.bar(labels, values, color=colors[0], alpha=0.8, edgecolor=colors[1], linewidth=2)
            ax.set_xlabel(x_col, fontsize=12, fontweight='bold')
            ax.set_ylabel(y_col, fontsize=12, fontweight='bold')

            # Add value labels on bars
            for bar, value in zip(bars, values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + max(values)*0.01,
                       f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

        elif chart_type == 'line':
            ax.plot(labels, values, color=colors[0], linewidth=3, marker='o',
                   markersize=8, markerfacecolor=colors[1], markeredgecolor=colors[0])
            ax.set_xlabel(x_col, fontsize=12, fontweight='bold')
            ax.set_ylabel(y_col, fontsize=12, fontweight='bold')
            ax.grid(True, alpha=0.3)

            # Add value labels on points
            for i, (label, value) in enumerate(zip(labels, values)):
                ax.annotate(f'{value:.1f}', (i, value), textcoords="offset points",
                           xytext=(0,10), ha='center', fontweight='bold')

        elif chart_type == 'pie':
            wedges, texts, autotexts = ax.pie(values, labels=labels, colors=colors[:len(values)],
                                            autopct='%1.1f%%', startangle=90,
                                            textprops={'fontsize': 10, 'fontweight': 'bold'})
            ax.axis('equal')

        # Set title
        ax.set_title(title, fontsize=16, fontweight='bold', pad=20)

        # Rotate x-axis labels if needed
        if chart_type in ['bar', 'line'] and len(labels) > 5:
            plt.xticks(rotation=45, ha='right')

        # Adjust layout
        plt.tight_layout()

        return fig

    except Exception as e:
        print(f"Chart creation error: {e}")
        return None

def generate_chart_config(data, chart_type, x_col, y_col, color_scheme, limit=10):
    """Generate chart configuration for frontend"""
    limited_data = data[:limit]

    labels = [str(row[x_col]) for row in limited_data]
    values = []

    for row in limited_data:
        try:
            values.append(float(row[y_col]))
        except:
            values.append(0)

    color_map = {
        "blue-green": {"bg": "rgba(0, 128, 128, 0.7)", "border": "rgba(0, 128, 128, 1)"},
        "blue": {"bg": "rgba(0, 100, 255, 0.7)", "border": "rgba(0, 100, 255, 1)"},
        "neutral": {"bg": "rgba(100, 100, 100, 0.7)", "border": "rgba(100, 100, 100, 1)"}
    }

    colors = color_map.get(color_scheme, color_map["blue"])

    return {
        'type': chart_type,
        'data': {
            'labels': labels,
            'datasets': [{
                'label': f'{y_col} by {x_col}',
                'data': values,
                'backgroundColor': colors["bg"],
                'borderColor': colors["border"],
                'borderWidth': 2
            }]
        },
        'options': {
            'responsive': True,
            'scales': {
                'y': {'beginAtZero': True}
            } if chart_type in ['bar', 'line'] else {}
        }
    }

def generate_infographic_prompt(data, x_col, y_col, title, tone, color_scheme, custom_prompt=None):
    """Generate AI prompt for infographic creation"""
    if custom_prompt:
        return custom_prompt

    # Analyze data for better prompts
    data_summary = f"data about {x_col} and {y_col}"
    if len(data) > 0:
        sample_values = [str(row[x_col]) for row in data[:3]]
        data_summary = f"data showing {x_col} categories: {', '.join(sample_values)}"

    tone_styles = {
        "formal": "professional, clean, corporate style",
        "casual": "friendly, approachable, modern style",
        "promotional": "eye-catching, vibrant, marketing style"
    }

    color_prompts = {
        "blue-green": "blue and teal color scheme, ocean-inspired colors",
        "blue": "blue color palette, professional blue tones",
        "neutral": "neutral colors, grayscale with subtle accents"
    }

    style = tone_styles.get(tone, "clean, professional style")
    colors = color_prompts.get(color_scheme, "blue color scheme")

    prompt = f"Modern infographic design, {style}, {colors}, data visualization, charts and graphs, {data_summary}, clean layout, white background, professional typography, high quality, detailed"

    return prompt

def create_infographic_overlay(base_image, data, x_col, y_col, title, tone, color_scheme, language):
    """Add data overlay to generated infographic and return PIL Image"""
    try:
        # Work with the PIL image directly
        image = base_image.convert("RGBA")
        draw = ImageDraw.Draw(image)

        try:
            font = ImageFont.truetype("arial.ttf", 20)
            title_font = ImageFont.truetype("arial.ttf", 24)
        except:
            font = ImageFont.load_default()
            title_font = ImageFont.load_default()

        color_map = {
            "blue-green": (0, 128, 128),
            "blue": (0, 0, 255),
            "neutral": (100, 100, 100)
        }
        text_color = color_map.get(color_scheme, (0, 0, 255))

        # Add title
        draw.text((50, 30), title, font=title_font, fill=text_color)

        # Add data points
        y_offset = 80
        for i, row in enumerate(data[:5]):  # Show top 5 data points
            text = f"{row[x_col]}: {row[y_col]}"
            draw.text((50, y_offset), text, font=font, fill=text_color)
            y_offset += 35

        return image
    except Exception as e:
        print(f"Overlay error: {e}")
        return None

# --- API Routes ---

@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'models_loaded': models_loaded,
        'timestamp': time.time()
    })

@app.route('/api/upload', methods=['POST'])
def upload_csv():
    """Upload and analyze CSV file"""
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file provided'}), 400

        file = request.files['file']
        if file.filename == '' or not allowed_file(file.filename):
            return jsonify({'error': 'Invalid file type. Only CSV files allowed'}), 400

        # Save file
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Analyze CSV
        df = pd.read_csv(filepath)
        numeric_cols, categorical_cols = detect_column_types(df)

        # Get sample data
        sample_data = df.head(5).to_dict(orient='records')

        return jsonify({
            'success': True,
            'filename': filename,
            'columns': df.columns.tolist(),
            'numeric_columns': numeric_cols,
            'categorical_columns': categorical_cols,
            'row_count': len(df),
            'sample_data': sample_data
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/chart-options', methods=['POST'])
def get_chart_options():
    """Get available chart options based on uploaded data"""
    try:
        data = request.get_json()
        filename = data.get('filename')

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        numeric_cols, categorical_cols = detect_column_types(df)

        # Suggest defaults
        x_default = categorical_cols[0] if categorical_cols else df.columns[0]
        y_default = numeric_cols[0] if numeric_cols else (df.columns[1] if len(df.columns) > 1 else df.columns[0])

        options = {
            'chart_types': [
                {'value': 'bar', 'label': 'Bar Chart'},
                {'value': 'line', 'label': 'Line Chart'},
                {'value': 'pie', 'label': 'Pie Chart'},
                {'value': 'infographic', 'label': 'AI Infographic'}
            ],
            'columns': df.columns.tolist(),
            'tones': [
                {'value': 'formal', 'label': 'Formal'},
                {'value': 'casual', 'label': 'Casual'},
                {'value': 'promotional', 'label': 'Promotional'}
            ],
            'color_schemes': [
                {'value': 'blue-green', 'label': 'Blue Green'},
                {'value': 'blue', 'label': 'Blue'},
                {'value': 'neutral', 'label': 'Neutral'}
            ],
            'languages': [
                {'value': 'en', 'label': 'English'},
                {'value': 'hi', 'label': 'Hindi'}
            ],
            'defaults': {
                'chart_type': 'bar',
                'x_axis': x_default,
                'y_axis': y_default,
                'tone': 'formal',
                'color_scheme': 'blue-green',
                'language': 'en'
            }
        }

        return jsonify({'success': True, 'options': options})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/generate', methods=['POST'])
def generate_visualization():
    """Generate chart or infographic based on parameters"""
    try:
        data = request.get_json()
        filename = data.get('filename')
        params = data.get('parameters', {})

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        # Load data
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        data_records = df.to_dict(orient='records')

        # Extract parameters
        chart_type = params.get('chart_type', 'bar')
        x_col = params.get('x_axis')
        y_col = params.get('y_axis')
        tone = params.get('tone', 'formal')
        color_scheme = params.get('color_scheme', 'blue-green')
        language = params.get('language', 'en')
        title = params.get('title', f'{y_col} by {x_col}')

        start_time = time.time()
        result = {'success': True}

        if chart_type == 'infographic' and models_loaded:
            # Generate AI infographic
            custom_prompt = params.get('custom_prompt')

            prompt = generate_infographic_prompt(
                data_records, x_col, y_col, title, tone, color_scheme, custom_prompt
            )

            print(f"🎨 Generated prompt: {prompt}")

            # Generate base image
            base_image = pipe(
                prompt,
                num_inference_steps=params.get('inference_steps', 25),
                guidance_scale=params.get('guidance_scale', 7.5),
                width=params.get('width', 512),
                height=params.get('height', 512)
            ).images[0]

            # Add data overlay
            final_image = create_infographic_overlay(
                base_image, data_records, x_col, y_col, title, tone, color_scheme, language
            )

            if final_image:
                # Convert to base64
                base64_image = pil_image_to_base64(final_image)

                if base64_image:
                    result['infographic'] = {
                        'base64': base64_image,
                        'prompt_used': prompt,
                        'format': 'PNG',
                        'dimensions': f"{final_image.width}x{final_image.height}"
                    }
                else:
                    return jsonify({'error': 'Failed to convert infographic to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate infographic'}), 500

        else:
            # Generate chart image using matplotlib
            chart_fig = create_chart_image(
                data_records, chart_type, x_col, y_col, title, color_scheme
            )

            if chart_fig:
                # Convert chart to base64
                chart_base64 = matplotlib_to_base64(chart_fig)

                if chart_base64:
                    result['chart'] = {
                        'base64': chart_base64,
                        'format': 'PNG',
                        'type': chart_type
                    }

                    # Also include chart config for frontend compatibility
                    result['chart_config'] = generate_chart_config(
                        data_records, chart_type, x_col, y_col, color_scheme
                    )
                else:
                    return jsonify({'error': 'Failed to convert chart to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate chart'}), 500

        # Add metrics
        result['metrics'] = {
            'generation_time': round(time.time() - start_time, 2),
            'cpu_usage': psutil.cpu_percent(),
            'memory_usage': psutil.virtual_memory().percent
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/generate-base64-only', methods=['POST'])
def generate_base64_only():
    """Generate visualization and return only base64 (no file saving)"""
    try:
        data = request.get_json()
        filename = data.get('filename')
        params = data.get('parameters', {})

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        # Load data
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)
        data_records = df.to_dict(orient='records')

        # Extract parameters
        chart_type = params.get('chart_type', 'bar')
        x_col = params.get('x_axis')
        y_col = params.get('y_axis')
        tone = params.get('tone', 'formal')
        color_scheme = params.get('color_scheme', 'blue-green')
        title = params.get('title', f'{y_col} by {x_col}')

        start_time = time.time()

        if chart_type == 'infographic' and models_loaded:
            # Generate AI infographic
            custom_prompt = params.get('custom_prompt')

            prompt = generate_infographic_prompt(
                data_records, x_col, y_col, title, tone, color_scheme, custom_prompt
            )

            print(f"🎨 Generated prompt: {prompt}")

            # Generate base image
            base_image = pipe(
                prompt,
                num_inference_steps=params.get('inference_steps', 25),
                guidance_scale=params.get('guidance_scale', 7.5),
                width=params.get('width', 512),
                height=params.get('height', 512)
            ).images[0]

            # Add data overlay
            final_image = create_infographic_overlay(
                base_image, data_records, x_col, y_col, title, tone, color_scheme, 'en'
            )

            if final_image:
                # Convert to base64
                base64_image = pil_image_to_base64(final_image)

                if base64_image:
                    return jsonify({
                        'success': True,
                        'image_base64': base64_image,
                        'prompt_used': prompt,
                        'format': 'PNG',
                        'type': 'infographic',
                        'dimensions': f"{final_image.width}x{final_image.height}",
                        'metrics': {
                            'generation_time': round(time.time() - start_time, 2),
                            'cpu_usage': psutil.cpu_percent(),
                            'memory_usage': psutil.virtual_memory().percent
                        }
                    })
                else:
                    return jsonify({'error': 'Failed to convert image to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate infographic'}), 500
        else:
            # Generate chart image
            chart_fig = create_chart_image(
                data_records, chart_type, x_col, y_col, title, color_scheme
            )

            if chart_fig:
                # Convert to base64
                base64_image = matplotlib_to_base64(chart_fig)

                if base64_image:
                    return jsonify({
                        'success': True,
                        'image_base64': base64_image,
                        'format': 'PNG',
                        'type': chart_type,
                        'metrics': {
                            'generation_time': round(time.time() - start_time, 2),
                            'cpu_usage': psutil.cpu_percent(),
                            'memory_usage': psutil.virtual_memory().percent
                        }
                    })
                else:
                    return jsonify({'error': 'Failed to convert chart to base64'}), 500
            else:
                return jsonify({'error': 'Failed to generate chart'}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/convert-to-base64', methods=['POST'])
def convert_existing_to_base64():
    """Convert existing saved image to base64"""
    try:
        data = request.get_json()
        filename = data.get('filename')

        if not filename:
            return jsonify({'error': 'Filename required'}), 400

        filepath = os.path.join(app.config['OUTPUT_FOLDER'], filename)

        if not os.path.exists(filepath):
            return jsonify({'error': 'File not found'}), 404

        base64_image = image_file_to_base64(filepath)

        if base64_image:
            return jsonify({
                'success': True,
                'image_base64': base64_image,
                'format': 'PNG'
            })
        else:
            return jsonify({'error': 'Failed to convert image to base64'}), 500

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/outputs/<filename>')
def serve_output(filename):
    """Serve generated output files (fallback for direct file access)"""
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename)

@app.route('/api/data/<filename>')
def get_data_preview(filename):
    """Get data preview for a specific file"""
    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        df = pd.read_csv(filepath)

        # Get basic stats
        stats = {
            'shape': df.shape,
            'columns': df.columns.tolist(),
            'dtypes': df.dtypes.to_dict(),
            'null_counts': df.isnull().sum().to_dict(),
            'sample_data': df.head(10).to_dict(orient='records')
        }

        # Add numeric column statistics
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            stats['numeric_stats'] = df[numeric_cols].describe().to_dict()

        return jsonify({'success': True, 'stats': stats})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/prompt-suggestions', methods=['POST'])
def get_prompt_suggestions():
    """Get AI prompt suggestions based on data"""
    try:
        data = request.get_json()
        x_col = data.get('x_axis')
        y_col = data.get('y_axis')
        tone = data.get('tone', 'formal')
        color_scheme = data.get('color_scheme', 'blue')
        data_type = data.get('data_type', 'general')

        suggestions = []

        # Base prompts by data type
        base_prompts = {
            'sales': f"Professional sales infographic showing {x_col} vs {y_col}, corporate style, charts and graphs",
            'marketing': f"Eye-catching marketing infographic, {x_col} and {y_col} data, vibrant design, call-to-action elements",
            'analytics': f"Clean analytics dashboard style, {x_col} by {y_col}, data visualization, KPI layout",
            'financial': f"Financial report infographic, {x_col} and {y_col} metrics, professional charts, business style",
            'general': f"Modern data infographic showing {x_col} vs {y_col}, clean layout, professional design"
        }

        # Add tone and color variations
        for data_cat, base in base_prompts.items():
            prompt = generate_infographic_prompt([], x_col, y_col, f"{x_col} Analysis", tone, color_scheme)
            suggestions.append({
                'category': data_cat,
                'prompt': prompt,
                'description': f"{data_cat.title()} style infographic"
            })

        return jsonify({
            'success': True,
            'suggestions': suggestions,
            'custom_parameters': {
                'inference_steps': {'min': 10, 'max': 50, 'default': 25},
                'guidance_scale': {'min': 1, 'max': 20, 'default': 7.5},
                'width': {'options': [512, 768, 1024], 'default': 512},
                'height': {'options': [512, 768, 1024], 'default': 512}
            }
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# --- Ngrok Configuration ---
NGROK_AUTH_TOKEN = "30JBYOiedMdItKFKaBrSQcQ82yo_4EtXeU17QiqfTpDAYSieA"
ngrok_tunnel = None

def setup_ngrok():
    """Setup ngrok tunnel"""
    global ngrok_tunnel
    try:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        ngrok_tunnel = ngrok.connect(5000)
        public_url = ngrok_tunnel.public_url
        print(f"🌐 Public URL: {public_url}")
        print(f"🔗 API Base URL: {public_url}/api")
        return public_url
    except Exception as e:
        print(f"❌ Ngrok setup failed: {e}")
        return None

def cleanup_ngrok():
    """Cleanup ngrok tunnel on shutdown"""
    global ngrok_tunnel
    try:
        if ngrok_tunnel:
            ngrok.disconnect(ngrok_tunnel.public_url)
        ngrok.kill()
        print("🔌 Ngrok tunnel closed")
    except:
        pass

# Register cleanup function
atexit.register(cleanup_ngrok)

@app.route('/api/tunnel-info', methods=['GET'])
def get_tunnel_info():
    """Get ngrok tunnel information"""
    global ngrok_tunnel
    if ngrok_tunnel:
        return jsonify({
            'success': True,
            'public_url': ngrok_tunnel.public_url,
            'api_base_url': f"{ngrok_tunnel.public_url}/api",
            'status': 'active'
        })
    else:
        return jsonify({
            'success': False,
            'status': 'inactive',
            'message': 'Ngrok tunnel not active'
        })

# --- Error Handlers ---
@app.errorhandler(413)
def too_large(e):
    return jsonify({'error': 'File too large. Maximum size is 16MB'}), 413

@app.errorhandler(404)
def not_found(e):
    return jsonify({'error': 'Endpoint not found'}), 404

@app.errorhandler(500)
def internal_error(e):
    return jsonify({'error': 'Internal server error'}), 500

# --- Run App ---
if __name__ == '__main__':
    print("🚀 Starting AI Infographic API Server...")
    print(f"🤖 Models loaded: {models_loaded}")

    # Setup ngrok tunnel
    public_url = setup_ngrok()

    print("\n📋 Available endpoints:")
    print("  POST /api/upload - Upload CSV file")
    print("  POST /api/chart-options - Get chart options")
    print("  POST /api/generate - Generate visualization (includes base64)")
    print("  POST /api/generate-base64-only - Generate only base64 image")
    print("  POST /api/convert-to-base64 - Convert existing file to base64")
    print("  POST /api/prompt-suggestions - Get AI prompt suggestions")
    print("  GET /api/health - Health check")
    print("  GET /api/tunnel-info - Ngrok tunnel info")
    print("  GET /api/data/<filename> - Data preview")
    print("  GET /api/outputs/<filename> - Serve generated files (fallback)")

    if public_url:
        print(f"\n🌍 Your API is accessible at: {public_url}/api")
        print(f"📖 Example: {public_url}/api/health")

    print("\n⚡ Server starting on http://localhost:5000")
    print("🔄 Use Ctrl+C to stop the server")

    try:
        app.run(host='0.0.0.0', port=5000, debug=False)
    except KeyboardInterrupt:
        print("\n🛑 Server stopped by user")
    except Exception as e:
        print(f"❌ Server error: {e}")
    finally:
        cleanup_ngrok()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:35:43] "POST /api/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:04] "POST /api/upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:04] "OPTIONS /api/chart-options HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:05] "POST /api/chart-options HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:16] "OPTIONS /api/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:17] "POST /api/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:30] "OPTIONS /api/generate HTTP/1.1" 200 -


🎨 Generated prompt: Modern infographic design, professional, clean, corporate style, blue color palette, professional blue tones, data visualization, charts and graphs, data showing Age categories: 71, 34, 80, clean layout, white background, professional typography, high quality, detailed


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:35] "POST /api/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:49] "OPTIONS /api/generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 09:38:51] "POST /api/generate HTTP/1.1" 200 -
